In [0]:
'''
RNN의 일종인 GRU를 이용, IMDB Dataset으로 문장 감정 분석을 해봅니다.
텍스트 형태의 Dataset인 IMDB Dataset은 50,000건의 영화 리뷰로 이루어져 있습니다.
각 리뷰는 다수의 영어 문장들로 이루어져 있으며, 평점이 7점 이상의 긍정적인 영화 리뷰는 2로, 
평점이 4점 이하인 부정적인 영화 리뷰는 1로 레이블링 되어 있습니다. 
영화 리뷰 텍스트를 RNN 에 입력시켜 영화평의 전체 내용을 압축하고, 
이렇게 압축된 리뷰가 긍정적인지 부정적인지 판단해주는 간단한 분류 모델을 만드는 것이 이번 프로젝트의 목표입니다.
'''

import os
import torch
import torch.nn as nn
import torch.nn.functional as F

from torchtext import data, datasets

import numpy as np

In [0]:
# 본인의 구글 드라이브 → 지금 실행중인 코드

# google.colab.drive : 구글 드라이브에서 파일을 가져오기 위한 코드를 담고 있다.
from google.colab import drive

# 본인의 구글 드라이브를 '/gdrive' 라는 경로로 하여 쓸 수 있다.
drive.mount('/gdrive', force_remount=True)

Mounted at /gdrive


In [0]:
# Hyperparameter
batch_size = 64
lr = 0.001
epochs = 20
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print("다음 기기로 학습합니다:", device)

다음 기기로 학습합니다: cuda:0


In [0]:
# 데이터 로딩하기
print("데이터 로딩중...")

text = data.Field(sequential=True, # 시계열 데이터임을 알림
                  batch_first=True, # (Batch_size, ?, ?)
                  lower=True) # 모든 대문자 -> 소문자 변환

label = data.Field(sequential=False, 
                   batch_first=True)

# IMDB를 text / label 로 나누어 위에 정의한 Field에 각각 나눠 담는다
trainset, testset = datasets.IMDB.splits(text, label)

# Vocabulary 형성
text.build_vocab(trainset,
                 min_freq=5)  # 5번 이상 등장한 단어만 token화하여 vocabulary에 추가

label.build_vocab(trainset)

# 학습용 데이터를 학습셋 80% 검증셋 20% 로 나누기
trainset, valset = trainset.split(split_ratio=0.8)

# 학습 시 반복문 활용을 위한 Iterator 지정
train_iter, val_iter, test_iter = data.BucketIterator.splits(
                                  (trainset, valset, testset), 
                                  batch_size=batch_size,
                                  shuffle=True, 
                                  repeat=False)
# Output Shape
n_classes = 2 # Positive / Negative

print("[학습셋]: %d [검증셋]: %d [테스트셋]: %d [단어수]: %d [클래스] %d"
      % (len(trainset),len(valset), len(testset), len(text.vocab), n_classes))

데이터 로딩중...
[학습셋]: 20000 [검증셋]: 5000 [테스트셋]: 25000 [단어수]: 46159 [클래스] 2


In [0]:
# Training Set의 첫번째 Text 및 Label 시각화
print(vars(trainset[0])['text'])
print(vars(trainset[0])['label'])
for i in range(10):
  print(len(vars(trainset[i])['text']))

['heart', 'pounding', 'erotic', 'drama', 'are', 'the', 'words', 'that', 'come', 'to', 'mind', 'when', 'i', 'think', 'of', '"secret', 'games".', 'it', 'becomes', 'more', 'erotic', 'as', 'the', 'film', 'goes', 'along', 'and', 'at', 'one', 'point', 'blew', 'me', 'away!', 'i', "didn't", 'expect', 'the', 'delightful', 'scene', 'i', 'was', 'about', 'to', 'encounter.', 'the', '"call', 'girl"', 'has', 'her', 'first', 'customer', 'and', 'what', 'a', 'customer!', 'one', 'of', 'the', 'most', 'erotic', 'lesbian', 'scenes', 'i', 'have', 'ever', 'seen.', 'the', 'husband', 'should', 'have', 'listened', 'to', 'his', 'wife', 'and', 'perhaps', 'she', "wouldn't", 'have', 'gone', 'on', 'this', 'erotic', 'journey.', 'it', 'turned', 'out', 'to', 'cost', 'them', 'in', 'the', 'end', 'but,', 'it', 'was', 'one', 'exciting', 'ride!', 'go', 'see', 'this', 'movie!!!']
pos
103
129
230
181
148
144
132
212
160
734


In [0]:
# Model 정의
class BasicGRU(nn.Module):
  def __init__(self, n_layers, hidden_dim, n_vocab, embed_dim, n_classes, dropout_p=0.2):
    super(BasicGRU, self).__init__()
    print("Building Basic GRU model...")
    self.n_layers = n_layers # Stacked RNN의 층 수 결정
    self.embed = nn.Embedding(n_vocab, embed_dim) # Vocab Embedding
    self.hidden_dim = hidden_dim # Dimension of Hidden state of RNN
    self.dropout = nn.Dropout(dropout_p) # Dropout

    # GRU Model 정의
    self.gru = nn.GRU(embed_dim, self.hidden_dim,
                      num_layers=self.n_layers,
                      batch_first=True)
    # 출력층 MLP
    self.out = nn.Linear(self.hidden_dim, n_classes)

  # RNN 내부 State 초기화
  def _init_state(self, batch_size=1):
    # Model의 parameter의 data를 weight로 뽑아온다
    weight = next(self.parameters()).data

    # 새로운 Tensor 생성 후 0으로 초기화
    return weight.new(self.n_layers, batch_size, self.hidden_dim).zero_()

  # Feed-Fowarding 함수
  def forward(self, x):
    x = self.embed(x) # Vocab -> Vector
    h_0 = self._init_state(batch_size=x.size(0))
        
    # GRU는 x(모델 출력), _ (hidden/cell state) 반환
    x, _ = self.gru(x, h_0)  # [i, b, h]

    # 출력 중 시간 상 맨 마지막 출력만 따온다
    h_t = x[:,-1,:]

    # Dropout 적용
    self.dropout(h_t)

    # 출력의 MLP층 적용, 출력 차원(2)로 맞춰준다
    pred = self.out(h_t)  # [b, h] -> [b, o]
    return pred

In [0]:
# 학습 함수 정의
def train(model, optimizer, train_iter):
    model.train() # Dropout 및 Gradient 계산 활성화

    # Iterator를 이용한 반복문
    for b, batch in enumerate(train_iter):
        # Batch 별로 Text / Label 각각을 학습 장치(GPU)로 보낸다
        x, y = batch.text.to(device), batch.label.to(device)

        # Label 값을 0과 1로 변환 (IMDB의 Label은 1 or 2 로 되어있습니다)
        # sub_ : sub() 함수의 In-place (자신에게 적용되는) version
        # sub(x) : x 값을 self에서 빼줍니다.
        y.data.sub_(1)

        optimizer.zero_grad() # Gradient 초기화
        logit = model(x) # Feed-Fowarding

        # Label과 Model 출력의 CEE 계산
        loss = F.cross_entropy(logit, y)

        loss.backward() # Gradient 계산
        optimizer.step() # Parameter Update

In [0]:
# 평가 함수 정의
def evaluate(model, val_iter):
    """evaluate model"""
    model.eval() # Dropout 및  Gradient 계산 중지

    # 덧셈 누적을 위한 변수
    # corrects : 맞은 숫자 누적
    # total_loss : Loss 값 누적
    corrects, total_loss = 0, 0

    # Iterator를 통한 Validation을 위한 반복문 실행
    for batch in val_iter:
        # 위와 동일
        x, y = batch.text.to(device), batch.label.to(device)
        y.data.sub_(1)
        out = model(x)

        # reduction='sum' : 출력이 모두 더해져 하나의 값으로 나온다
        loss = F.cross_entropy(out, y, reduction='sum')
        
        total_loss += loss.item() # Loss 누적

        # out : tensor.size = ([batch_size, 2])
        # out.max(1) : out.max(1)[0] : 2개 값 중 최댓값의 Index가 1인 경우의 값 리스트
        #                out.max(1)[1] : 최댓값의 Index가 1인 경우 1, 아닌 경우 0
        # (out.max(1)[1].data == y.data) : 해당 Index의 Label과 Model 출력이 같으면 True, 다르면 False : ([batch_size,])
        # (out.max(1)[1].data == y.data).sum() : True 인 값의 갯수
        corrects += (out.max(1)[1].data == y.data).sum() # 정답 수 누적
        
    # Loss 정규화
    size = len(val_iter.dataset)
    avg_loss = total_loss / size

    # Accuracy 계산
    avg_accuracy = 100.0 * corrects / size
    return avg_loss, avg_accuracy

In [0]:
a = torch.ones((10, 2))
for i in a:
  i[0] = 0

b = torch.ones((1, 10))
print(b, '\n')

print(a, '\n')
print(a.max(1), '\n')
print(a.max(1)[1])
print(a.max(1)[1].data)
print(a.max(1)[1].data == b.data)
print((a.max(1)[1].data == b.data).sum())

tensor([[1., 1., 1., 1., 1., 1., 1., 1., 1., 1.]]) 

tensor([[0., 1.],
        [0., 1.],
        [0., 1.],
        [0., 1.],
        [0., 1.],
        [0., 1.],
        [0., 1.],
        [0., 1.],
        [0., 1.],
        [0., 1.]]) 

torch.return_types.max(
values=tensor([1., 1., 1., 1., 1., 1., 1., 1., 1., 1.]),
indices=tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1])) 

tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1])
tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1])
tensor([[True, True, True, True, True, True, True, True, True, True]])
tensor(10)


In [0]:
# Model 객체화
model = BasicGRU(1, 256, vocab_size, 128, n_classes, 0.5).to(device)
optimizer = torch.optim.Adam(model.parameters())

Building Basic GRU model...


In [0]:
best_val_loss = None

# Epoch 동안 학습 진행
for e in range(1, epochs+1):
    train(model, optimizer, train_iter) # 학습 진행
    val_loss, val_accuracy = evaluate(model, val_iter) # 검증 진행

    # 진행 결과 출력
    print("[Epoch %d] Val_Loss :%5.2f | Val_Acc :%5.2f" % (e, val_loss, val_accuracy))
    
    # Validation Loss가 가장 적은 최적의 모델을 저장
    if not best_val_loss or val_loss < best_val_loss:
        torch.save(model.state_dict(), '/gdrive/My Drive/RNN_IMDB.pt')
        best_val_loss = val_loss

[Epoch 1] Val_Loss : 0.69 | Val_Acc :50.20
[Epoch 2] Val_Loss : 0.69 | Val_Acc :51.86
[Epoch 3] Val_Loss : 0.67 | Val_Acc :59.46
[Epoch 4] Val_Loss : 0.35 | Val_Acc :85.60
[Epoch 5] Val_Loss : 0.32 | Val_Acc :86.90
[Epoch 6] Val_Loss : 0.35 | Val_Acc :87.18
[Epoch 7] Val_Loss : 0.39 | Val_Acc :87.82
[Epoch 8] Val_Loss : 0.47 | Val_Acc :86.16
[Epoch 9] Val_Loss : 0.48 | Val_Acc :87.02
[Epoch 10] Val_Loss : 0.51 | Val_Acc :86.70
[Epoch 11] Val_Loss : 0.56 | Val_Acc :86.94
[Epoch 12] Val_Loss : 0.51 | Val_Acc :87.24
[Epoch 13] Val_Loss : 0.54 | Val_Acc :86.68
[Epoch 14] Val_Loss : 0.56 | Val_Acc :87.42
[Epoch 15] Val_Loss : 0.59 | Val_Acc :87.62
[Epoch 16] Val_Loss : 0.63 | Val_Acc :87.44
[Epoch 17] Val_Loss : 0.65 | Val_Acc :87.52
[Epoch 18] Val_Loss : 0.68 | Val_Acc :87.52
[Epoch 19] Val_Loss : 0.70 | Val_Acc :87.42
[Epoch 20] Val_Loss : 0.72 | Val_Acc :87.54


In [0]:
# Test
model.load_state_dict(torch.load('/gdrive/My Drive/RNN_IMDB.pt'))
test_loss, test_acc = evaluate(model, test_iter)
print('테스트 오차: %5.2f | 테스트 정확도: %5.2f' % (test_loss, test_acc))

# 시각화
model.eval() # Dropout 및 Gradient 계산 중지

Class = ['Negative', 'Positive']
Token_to_String = [] # Token을 문자열로 바꾼 뒤 담을 리스트

# Iterator를 통한 Test를 위한 반복문 한번만 실행
for batch in test_iter:

  # 위와 동일
  x, y = batch.text.to(device), batch.label.to(device)
  y.data.sub_(1)
  out = model(x)

  # reduction='sum' : 출력이 모두 더해져 하나의 값으로 나온다
  loss = F.cross_entropy(out, y, reduction='sum')

  # out.max(1)[1] (Tensor, GPU) -> .data (Gradient 계산 및 1인 차원 제거, Tensor, GPU) -> (Tensor, CPU) -> (ndarray, CPU)
  pred_batch = out.max(1)[1].data.cpu().numpy() # Batch 단위 Model 출력 값
  ans_batch = y.data.cpu().numpy() # Batch 단위 Label 값

  for i in range(batch_size):
    tok_to_str = [] # 문장 단위로 담을 리스트
    for token in x[i]: 
      # x가 갖는 Token 각각을 원래 단어로 바꿔준다
      tok_to_str.append(text.vocab.itos[token])

    # 문장을 전체 리스트에 추가
    Token_to_String.append(tok_to_str) 

  # 반복문 탈출 : 반복문을 딱 한번 실행
  break 

for i in range(batch_size):
    print(Token_to_String[i])
    print('Answer :', Class[ans_batch[i]])
    print('Prediction :', Class[pred_batch[i]])

테스트 오차:  0.33 | 테스트 정확도: 86.32
["i've", 'seen', 'soap', 'operas', 'more', 'intelligent', 'than', 'this', 'movie.', 'bad', 'characters,', 'bad', 'story', 'and', 'bad', 'acting.', 'it', 'would', 'be', 'a', 'love', 'story', 'between', 'a', 'man', 'and', 'a', 'mermaid.', 'really', 'awful.']
Answer : Negative
Prediction : Negative
['this', 'movie', 'is', 'based', 'on', 'the', 'novel', 'island', 'of', 'dr.', 'moreau', 'by', 'h.g.', '<unk>', "it's", 'a', 'fairly', 'good', 'one', 'too,', "it's", 'at', 'least', 'better', 'than', 'the', 'version', 'by', 'john', '<unk>']
Answer : Positive
Prediction : Positive
['albert', 'pyun', 'delivers', 'a', 'very', 'good', '<unk>', 'about', 'a', 'junkie', 'who', 'tries', 'to', 'rip-off', 'a', 'big', '<unk>', 'a', 'lot', 'of', 'style', 'and', 'many', 'very', 'cool', 'actors.', 'burt', '<unk>', 'is', 'excellent.']
Answer : Positive
Prediction : Positive
['complete', 'entertainment!', 'although', 'there', 'are', 'many', 'strange', 'things', 'in', 'the', 'movie'

In [0]:
# 모델은 pt 파일 형태로 저장됩니다
torch.save(model.state_dict(), '/gdrive/My Drive/IMDB_RNN.pt')

# 모델을 불러오기 위해 지워줍니다
del model

model = MLP(784).to(device)
model.load_state_dict(torch.load('/gdrive/My Drive/IMDB_RNN.pt'))
model.to(device)